In [54]:
#!pip install -q arxiv
#%pip install -q arxiv
pass

In [65]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
#
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
#from langchain_openai import OpenAIEmbeddings
#from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain import hub
from langchain.agents import create_openai_tools_agent
from langchain.agents import AgentExecutor
#
import os
from dotenv import load_dotenv

In [10]:
load_dotenv()
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

### Заменить OpenAIEmbeddings

In [12]:
model_id = 'sentence-transformers/all-MiniLM-L6-v2'
#model_kwargs = {'device': 'cuda'}
model_kwargs = {'device': 'cpu'}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_id,
    model_kwargs=model_kwargs
)

e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [42]:
#loader = WebBaseLoader('https://docs.smith.langchain.com/')
loader = WebBaseLoader('https://huggingface.co/learn/nlp-course/ru/chapter1/2')
docs = loader.load()

# Загруженную HTML-страницу сохраняем в векторной базе данных. И создаем retriever
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
#vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
vectordb = FAISS.from_documents(documents, hf_embeddings)
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000268071DBA10>)

In [43]:
retriever_tool = create_retriever_tool(
    retriever, 'huggingface_nlp_course',
    'Поиск информации о huggingface. Для всех вопросов о huggingface по  NLP, ты должен использовать этот инструмент!'
)
retriever_tool.name

'huggingface_nlp_course'

In [44]:
## Wiki Tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)

wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.name

'wikipedia'

In [45]:
# Аналог wiki для научных статей
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)

arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [69]:
# Порядок поиска информации для агентов:
# сначала в wiki, затем в arxiv, затем в retriever_tool
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\Progi\\PythonProg\\NLP\\LangChain\\KrishNaik\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='huggingface_nlp_course', description='Поиск информации о huggingface. Для всех вопросов о huggingface по  NLP, ты должен использовать этот инструмент!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000268472C9300>, retriever=VectorStoreRetriever(

In [47]:
#llm = Ollama(model='sutyrin/saiga_mistral_7b')
llm = ChatOllama(model='sutyrin/saiga_mistral_7b')
llm

ChatOllama(model='sutyrin/saiga_mistral_7b')

In [66]:
# Get the prompt to use - you can modify this!
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

e:\Progi\PythonProg\NLP\LangChain\KrishNaik\.venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [68]:
# Агенты отвечают за последовательность действий для генерации ответа
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [64]:
answer = agent_executor.invoke({'input': 'Список типичных NLP-задач в huggingface'})
answer



> Entering new AgentExecutor chain...
В Hugging Face есть множество моделей, которые могут быть использованы для решения различных задач. Вот некоторые из них:

1. **Классификация текста**: Модель классифицирует текст на основе его категории (например, спам или неспам).
2. **Разметка текста**: Модель выделяет определенные сущности в тексте (например, имена людей, даты и т.д.).
3. **Получение ответов на вопросы**: Модель генерирует ответ на заданный вопрос.
4. **Перевод текста**: Модель переводит текст с одного языка на другой.
5. **Синтез текста**: Модель создает новый текст, основанный на данных, предоставленных пользователем.
6. **Получение рекомендаций**: Модель предлагает пользователю соответствующие ему товары или контент.
7. **Определение эмоций**: Модель определяет эмоцию, которую выражает текст (например, радость, грусть и т.д.).
8. **Разметка сентиментов**: Модель определяет положительный или негативный тон текста.
9. **Получение релевантных документов**: Модель выдает докум

{'input': 'Список типичных NLP-задач в huggingface',
 'output': 'В Hugging Face есть множество моделей, которые могут быть использованы для решения различных задач. Вот некоторые из них:\n\n1. **Классификация текста**: Модель классифицирует текст на основе его категории (например, спам или неспам).\n2. **Разметка текста**: Модель выделяет определенные сущности в тексте (например, имена людей, даты и т.д.).\n3. **Получение ответов на вопросы**: Модель генерирует ответ на заданный вопрос.\n4. **Перевод текста**: Модель переводит текст с одного языка на другой.\n5. **Синтез текста**: Модель создает новый текст, основанный на данных, предоставленных пользователем.\n6. **Получение рекомендаций**: Модель предлагает пользователю соответствующие ему товары или контент.\n7. **Определение эмоций**: Модель определяет эмоцию, которую выражает текст (например, радость, грусть и т.д.).\n8. **Разметка сентиментов**: Модель определяет положительный или негативный тон текста.\n9. **Получение релевантны

In [59]:
answer['output']

'В Hugging Face есть несколько задач, которые могут быть использованы для обучения и тестирования моделей на основе предобученных моделей. Вот некоторые из них:\n\n1. **Classification**: Задача классификации предполагает, что модель должна определить категорию или класс для данного текста. Например, можно обучить модель на тексте с рецензиями и затем использовать ее для определения положительных или негативных отзывов.\n2. **Sequence Classification**: Задача секвенциальной классификации похожа на обычную классификацию, но вместо того чтобы работать с отдельными текстами, модель должна анализировать последовательность слов или токенов. Например, можно обучить модель на серии слов и затем использовать ее для определения эмоций в тексте.\n3. **Text Generation**: Задача генерации текста предполагает, что модель должна создавать новый текст на основе данных, которые она получила. Например, можно обучить модель на серии слов и затем использовать ее для создания нового текста в стиле оригинал

In [60]:
answer = agent_executor.invoke({'input': 'Почему NLP в huggingface это сложно?'})
answer



> Entering new AgentExecutor chain...
Вот несколько причин, почему NLP в Hugging Face может быть сложным:

1. **Огромное количество моделей**: Hugging Face предоставляет более чем 200 различных моделей NLP, что может сделать выбор модели для конкретной задачи сложным.

2. **Необходимость понимания архитектуры и параметров моделей**: Для правильного использования модели NLP необходимо понимание, как она работает и какие параметры можно изменить для достижения лучших результатов.

3. **Оптимизация гиперпараметров**: Оптимальные значения гиперпараметров могут варьироваться в зависимости от конкретной задачи и данных, что может потребовать некоторых экспериментов для их настройки.

4. **Оптимизация вычислительных ресурсов**: Некоторые модели NLP требуют большого количества вычислительных ресурсов, что может быть дорогостоящим и затратным в энергетическом отношении.

5. **Оптимизация памяти**: Некоторые модели NLP требуют большого объема памяти для работы, что может быть проблемой на неко

{'input': 'Почему NLP в huggingface это сложно?',
 'output': 'Вот несколько причин, почему NLP в Hugging Face может быть сложным:\n\n1. **Огромное количество моделей**: Hugging Face предоставляет более чем 200 различных моделей NLP, что может сделать выбор модели для конкретной задачи сложным.\n\n2. **Необходимость понимания архитектуры и параметров моделей**: Для правильного использования модели NLP необходимо понимание, как она работает и какие параметры можно изменить для достижения лучших результатов.\n\n3. **Оптимизация гиперпараметров**: Оптимальные значения гиперпараметров могут варьироваться в зависимости от конкретной задачи и данных, что может потребовать некоторых экспериментов для их настройки.\n\n4. **Оптимизация вычислительных ресурсов**: Некоторые модели NLP требуют большого количества вычислительных ресурсов, что может быть дорогостоящим и затратным в энергетическом отношении.\n\n5. **Оптимизация памяти**: Некоторые модели NLP требуют большого объема памяти для работы, 